**SLEEP CALL**

### 12S18027 Christopher A Hutabarat
### 12S18028 Alda G M Lumban Gaol
### 12S18032 Sarah H M Siahaan 

# **Analysis Emotion Of Comment Topic Reddit With Multi-label Classification Using SVM Algorithm**


In [1]:
import numpy as np, pandas as pd
import re
import nltk
import string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pandas import DataFrame as df
pd.options.mode.chained_assignment = None

In [2]:
df_go1 = pd.read_csv('goemotions_1.csv')
df_go2 = pd.read_csv('goemotions_2.csv')
df_go3 = pd.read_csv('goemotions_3.csv')

df_go_emotion = pd.concat([df_go1, df_go2, df_go3])

df_go_emotion.describe()

,created_utc,rater_id,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
count,2.112250e+05,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,...,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000
mean,1.547659e+09,37.619742,0.081103,0.043768,0.038272,0.064472,0.083418,0.028401,0.034840,0.045885,...,0.038779,0.008569,0.041259,0.006164,0.041591,0.006102,0.011954,0.031994,0.026105,0.261797
std,7.696842e+05,23.070528,0.272994,0.204580,0.191853,0.245592,0.276514,0.166116,0.183374,0.209236,...,0.193067,0.092172,0.198890,0.078269,0.199653,0.077880,0.108679,0.175985,0.159448,0.439614
min,1.546301e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.546993e+09,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.547664e+09,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.548320e+09,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.548979e+09,81.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# **Data Cleaning**

### **1. Drop kolom yang tidak dibutuhkan**

In [3]:
to_drop = ['author', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear']
df_go_emotion.drop(columns=to_drop, inplace=True)

df.head(df_go_emotion)

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,nrl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,confessions,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,facepalm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,starwarsspeculation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### **2. Cek data yang kosong (Missing Value)**

In [4]:
df_go_emotion.isna()

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71220,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71221,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71222,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71223,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### **3. Cek keunikan id sebagai index pada dataset**



In [5]:
df_go_emotion['id'].is_unique

False

### **4. Menghapus nilai yang sama pada dataset**



In [6]:
df_go_emotion.drop_duplicates(subset='id').reset_index(drop=True)

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,nrl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,confessions,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,facepalm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,starwarsspeculation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58006,He called [NAME] to the Steelers and was outsp...,eed7qdq,steelers,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
58007,I didn’t even notice that one. I noticed the b...,ef0lnq5,AnimalsBeingBros,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
58008,"Thanks, [NAME]",edirq0m,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58009,Everyone likes [NAME].,ee6pagw,heroesofthestorm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# **Text Preprocessing** "Jangan di eksekusi dulu sampe word tokenization"

### **1. Removal of Punctuations & Word Tokenization**

In [7]:
# Fungsi Punctuatuion Removal & Word Tokenization yang akan digunakan saat implementasi Model

import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

def tokenize(s):
    return re_tok.sub(r' \1 ', s).split()

### **2. Stopwords Removal**

In [8]:
# from nltk.corpus import stopwords
# nltk.download('stopwords') #in case stopwords is undefined

# STOPWORDS = set(stopwords.words('english'))
# def remove_stopwords(text):
#     """custom function to remove the stopwords"""
#     return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# df_go_emotion["text"] = df_go_emotion["text"].apply(lambda text: remove_stopwords(text))

# df_go_emotion.head()

### **3. Stemming**

In [9]:
# from nltk.stem.porter import PorterStemmer

# stemmer = PorterStemmer()
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])

# df_go_emotion["text"] = df_go_emotion["text"].apply(lambda text: stem_words(text))

# df_go_emotion.head()

### **4. Lemmatization**

In [10]:
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()
# def lemmatize_words(text):
#     return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

# df_go_emotion["text"] = df_go_emotion["text"].apply(lambda text: lemmatize_words(text))

# df_go_emotion.head()

# **Feature Extraction**

In [11]:
#Split data train and data test

from sklearn.model_selection import train_test_split

df = df_go_emotion

train, test = train_test_split(df, test_size=0.2)

to_drop = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
test.drop(columns=to_drop, inplace=True)

train.to_csv('go_emotion_train.csv', index=False)
test.to_csv('go_emotion_test.csv', index=False)

In [12]:
COMMENT = 'text'

train = pd.read_csv('go_emotion_train.csv')
test = pd.read_csv('go_emotion_test.csv')
subm = pd.read_csv('go_sample_result.csv')

In [13]:
label_cols = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,none
count,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,...,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000,168980.000000
mean,0.080939,0.043757,0.038383,0.064416,0.083241,0.028062,0.034951,0.046047,0.018138,0.040058,...,0.008534,0.041218,0.006267,0.041697,0.006119,0.011990,0.032104,0.026411,0.261771,0.016221
std,0.272742,0.204554,0.192120,0.245493,0.276246,0.165152,0.183656,0.209587,0.133452,0.196096,...,0.091983,0.198794,0.078916,0.199897,0.077985,0.108839,0.176278,0.160356,0.439600,0.126324
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
# TF-IDF Feature Extraction

n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)
trn_term_doc = vec.fit_transform(train[COMMENT])

test_term_doc = vec.transform(test[COMMENT])

In [15]:
trn_term_doc, test_term_doc

(<168980x226898 sparse matrix of type '<class 'numpy.float64'>'
 	with 4910597 stored elements in Compressed Sparse Row format>,
 <42245x226898 sparse matrix of type '<class 'numpy.float64'>'
 	with 1154682 stored elements in Compressed Sparse Row format>)

In [16]:
# Naive Bayes Feature Equation

def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [17]:
x = trn_term_doc
test_x = test_term_doc

In [18]:
# NB-SVM Model with Logistic Regression

def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True, solver='liblinear')
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [19]:
# Implementasi Model

preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit admiration
fit amusement


C:\Users\toto\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


fit anger


C:\Users\toto\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


fit annoyance
fit approval
fit caring


...

fit sadness


C:\Users\toto\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


fit surprise


C:\Users\toto\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


fit neutral


In [20]:
# Mengolah data yang dihasilkan dari Model

submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)

submission.loc[submission['admiration'] > 0.5, 'admiration'] = 1
submission.loc[submission['amusement'] > 0.5, 'amusement'] = 1
submission.loc[submission['anger'] > 0.5, 'anger'] = 1
submission.loc[submission['annoyance'] > 0.5, 'annoyance'] = 1
submission.loc[submission['approval'] > 0.5, 'approval'] = 1
submission.loc[submission['caring'] > 0.5, 'caring'] = 1
submission.loc[submission['confusion'] > 0.5, 'confusion'] = 1
submission.loc[submission['curiosity'] > 0.5, 'curiosity'] = 1
submission.loc[submission['desire'] > 0.5, 'desire'] = 1
submission.loc[submission['disappointment'] > 0.5, 'disappointment'] = 1
submission.loc[submission['disapproval'] > 0.5, 'disapproval'] = 1
submission.loc[submission['disgust'] > 0.5, 'disgust'] = 1
submission.loc[submission['embarrassment'] > 0.5, 'embarrassment'] = 1
submission.loc[submission['excitement'] > 0.5, 'excitement'] = 1
submission.loc[submission['fear'] > 0.5, 'fear'] = 1
submission.loc[submission['gratitude'] > 0.5, 'gratitude'] = 1
submission.loc[submission['grief'] > 0.5, 'grief'] = 1
submission.loc[submission['joy'] > 0.5, 'joy'] = 1
submission.loc[submission['love'] > 0.5, 'love'] = 1
submission.loc[submission['nervousness'] > 0.5, 'nervousness'] = 1
submission.loc[submission['optimism'] > 0.5, 'optimism'] = 1
submission.loc[submission['pride'] > 0.5, 'pride'] = 1
submission.loc[submission['realization'] > 0.5, 'realization'] = 1
submission.loc[submission['relief'] > 0.5, 'relief'] = 1
submission.loc[submission['remorse'] > 0.5, 'remorse'] = 1
submission.loc[submission['sadness'] > 0.5, 'sadness'] = 1
submission.loc[submission['surprise'] > 0.5, 'surprise'] = 1
submission.loc[submission['neutral'] > 0.5, 'neutral'] = 1

submission.loc[submission['admiration'] < 0.5, 'admiration'] = 0
submission.loc[submission['amusement'] < 0.5, 'amusement'] = 0
submission.loc[submission['anger'] < 0.5, 'anger'] = 0
submission.loc[submission['annoyance'] < 0.5, 'annoyance'] = 0
submission.loc[submission['approval'] < 0.5, 'approval'] = 0
submission.loc[submission['caring'] < 0.5, 'caring'] = 0
submission.loc[submission['confusion'] < 0.5, 'confusion'] = 0
submission.loc[submission['curiosity'] < 0.5, 'curiosity'] = 0
submission.loc[submission['desire'] < 0.5, 'desire'] = 0
submission.loc[submission['disappointment'] < 0.5, 'disappointment'] = 0
submission.loc[submission['disapproval'] < 0.5, 'disapproval'] = 0
submission.loc[submission['disgust'] < 0.5, 'disgust'] = 0
submission.loc[submission['embarrassment'] < 0.5, 'embarrassment'] = 0
submission.loc[submission['excitement'] < 0.5, 'excitement'] = 0
submission.loc[submission['fear'] < 0.5, 'fear'] = 0
submission.loc[submission['gratitude'] < 0.5, 'gratitude'] = 0
submission.loc[submission['grief'] < 0.5, 'grief'] = 0
submission.loc[submission['joy'] < 0.5, 'joy'] = 0
submission.loc[submission['love'] < 0.5, 'love'] = 0
submission.loc[submission['nervousness'] < 0.5, 'nervousness'] = 0
submission.loc[submission['optimism'] < 0.5, 'optimism'] = 0
submission.loc[submission['pride'] < 0.5, 'pride'] = 0
submission.loc[submission['realization'] < 0.5, 'realization'] = 0
submission.loc[submission['relief'] < 0.5, 'relief'] = 0
submission.loc[submission['remorse'] < 0.5, 'remorse'] = 0
submission.loc[submission['sadness'] < 0.5, 'sadness'] = 0
submission.loc[submission['surprise'] < 0.5, 'surprise'] = 0
submission.loc[submission['neutral'] < 0.5, 'neutral'] = 0

submission.to_csv('go_emotion_svm_result.csv', index=False)

In [21]:
# Membuat Confusion Matrix dari data prediksi

from sklearn.metrics import confusion_matrix

y_test = submission
y_pred = subm

y_test.drop(columns='id', inplace=True)
y_pred.drop(columns='id', inplace=True)

cf_matrix = confusion_matrix(y_test.values.argmax(axis=1), y_pred.values.argmax(axis=1))

print(cf_matrix)

[[2347  944  960 1344 1788  581  757  921  313  788  988  386  187  510
   282  988   59  609  605   91  576   72  573   93  177  400  410 6359]
 [ 125   70   71   82   95   40   48   38   13   42   43   21   10   29
    19   51    4   37   27   10   30    7   32    6   11   23   27  361]
 [  67   24   33   50   52   17   25   22    8   21   26   12    6   20
     9   24    2   14   20    4   19    3   21    4    2   11    8  167]
 [  92   21   28   36   47   21   24   21    6   17   36   13   12   14
     7   34    1   16   27    4   12    3    9    7   10   11   11  193]
 [  77   38   29   48   66   22   33   26   12   27   39   17   11   21
    10   32    3   21   19    4   22    1   11    5    8   17   10  266]
 [  33   14   18   28   35    8    4   12    7   17   27    9    2   12
     4   28    2   11   11    3   10    1   11    1    8    6   14  125]
 [  55   25   18   31   37   15   21   10   11   24   25   10    8   10
     6   17    0   21   13    5   17    1    5    1    3  

In [22]:
# Menghitung akurasi dari Model

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.045354479820097056
